In [1]:
import sys
sys.path.insert(0,'..')

import numpy as np
import pandas as pd

import keras 
import keras.backend as K
from keras.optimizers import Adam

from gan_utils import *
from strudel_utils import *
from networks import *

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
config.gpu_options.visible_device_list = "0"
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))


gen_shape = (1, 20736, 1)
gen_inputs, gen_encoded, gen_outputs, gen_outputs_lambda = generator(gen_shape)

dis_inputs, dis_outputs = discriminator(K.int_shape(gen_outputs)[1:])

class_intermediate_input_shapes = [(K.int_shape(gen_inputs)[1], K.int_shape(gen_inputs)[2]//(2**i), [32,32,32,32,64,64,64,64][i]) for i in range(8)]
class_in, class_out = classifier(K.int_shape(gen_inputs)[1:], class_intermediate_input_shapes, add_in=False)


Using TensorFlow backend.
/home/elad/anaconda2/envs/TF-1.1/lib/python2.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/home/elad/anaconda2/envs/TF-1.1/lib/python2.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
x, y, has_transits = load_data()

classifier = Model(class_in, class_out, 'classifier')
classifier.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
# classifier.summary()
print('loaded Classifier')

log_callback = log_results_classifier(1, 'scores_classifier_only.txt')

classifier.fit(x, has_transits, epochs=16, batch_size=32, callbacks=[log_callback])
del x
del y
del has_transits

In [2]:
x = np.expand_dims(np.load('../total_x_sim.npy', mmap_mode='r'),axis=1)
x = np.pad(x, ((0, 0), (0, 0), (0, 30 + 96), (0, 0)), 'constant', constant_values=(0, 0))
classifier = keras.models.load_model('best_classifier_only_test.h5')

y_test = np.load('../total_params_sim.npy')[:,1] > 0
y_test = y_test[20000:]
y_pred = classifier.predict(x[20000:], verbose=1)[:,0]
fpr, tpr, thresholds_keras = roc_curve(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

thresh = thresholds_keras[find_nearest(fpr, 0.01)]
rate = tpr[find_nearest(fpr, 0.01)]
print(roc_auc)

179980/179980 [==============================] - 369s   
0.9554745951419907


In [3]:
fpr, tpr, thresholds_keras = roc_curve(y_test, y_pred)
d = {'tpr': tpr, 'fpr': fpr, 'thresholds': thresholds_keras}
roc = pd.DataFrame(data=d)
roc.to_csv('ROC_AUC_FULL_only_classfier.csv')